In [1]:
import  requests
from parsel import Selector

# 一 网络请求

In [2]:
# url为搜索的链接
url = 'https://www.instagram.com/explore/tags/%E4%BB%A3%E8%B4%AD/'

# 请求头 包含很多数据 
headers = {
'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
'accept-encoding':'gzip, deflate, sdch, br',
'accept-language':'zh-CN,zh;q=0.8',
'cache-control':'max-age=0',
'cookie':'mid=WUTYkQALAAEBhxrseb7HgMYHooES; _js_datr=rzNfWTLhNXuuwNgSQ_jeci8B; sessionid=IGSC81bfd268b72e3ae971952fb77e11f96706981fac28e770e6ac95e672e0377bd0%3A1VhxZct7qrV11FXWgFFCrV9p5Rz8ESJ4%3A%7B%22_auth_user_id%22%3A5690065567%2C%22_auth_user_backend%22%3A%22accounts.backends.CaseInsensitiveModelBackend%22%2C%22_auth_user_hash%22%3A%22%22%2C%22_token_ver%22%3A2%2C%22_token%22%3A%225690065567%3ASoNBfmOnivtqgH2ER5qBsmEIaPm6gJA8%3A7eca80b7a7592293f0f5896523f35dd3981429b133871bb69e20bef2a4a96309%22%2C%22_platform%22%3A4%2C%22last_refreshed%22%3A1499501508.9412579536%2C%22asns%22%3A%7B%22time%22%3A1499501507%2C%2247.74.12.143%22%3A45102%7D%7D; rur=ASH; csrftoken=wysnnnpLFItwvwi43teN0HYD3bNpLlr0; ds_user_id=5690065567; ig_pr=1; ig_vw=688',
'upgrade-insecure-requests':'1',
'user-agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36'
}

# 执行网络请求
z = requests.get(url,headers=headers)
# 查看状态
print z.status_code

200


# 二 解析网络请求返回的html

In [3]:
import js2xml

sel1 = Selector(text = z.text)
sel1.xpath('//script/text()')

# 页面是由javascript渲染的
# 找出返回图片列表的js代码
jscode = sel1.xpath("//script[contains(., 'window._sharedData')]/text()").extract_first()

# 将JavaScript代码转换成xml格式
parsed_js = js2xml.parse(jscode)
last = '"'+ parsed_js.xpath('//property[@name="end_cursor"]/string/text()')[0] + '"'
print last

"J0HWXAaKgAAAF0HWW_E5gAAAFiYA"


# 三 获取shortcode(短号)
- 每张图片的详情界面有一个专门的标识符,这个标识符就是shortcode(短号),通过这个短号就可知直接访问图片详情
- 如何收集短号靠的是不断地发送上拉刷新的请求,服务器会返回Json数据,从Json数据中就可以提取出新加载的图片的短号.

In [4]:
# 查询的基本地址
url_more = 'https://www.instagram.com/graphql/query/?'

# 从返回的信息中获取是否还有下一页
has_next_page = True

# 统计收集的图片数量
count = 0

# 收集短号的列表
shortcode_list = [];
while has_next_page == True:
    # 网络请求的参数
    # tag_name: 搜索关键词
    # first: 显示热门搜索的图片的数量
    # after: 上次加载的最后一张图片的游标
    # query_id: 搜索id 关键词tag_name对应的查询id
    params = {
        'variables':'{"tag_name":"代购","first":8,"after":'+last+'}',
        'query_id':'17875800862117404'
    }
    headers_more = {
        'accept':'*/*',
        'accept-encoding':'gzip, deflate, sdch, br',
        'accept-language':'zh-CN,zh;q=0.8',
        'cookie':'mid=WUTYkQALAAEBhxrseb7HgMYHooES; _js_datr=rzNfWTLhNXuuwNgSQ_jeci8B; sessionid=IGSC81bfd268b72e3ae971952fb77e11f96706981fac28e770e6ac95e672e0377bd0%3A1VhxZct7qrV11FXWgFFCrV9p5Rz8ESJ4%3A%7B%22_auth_user_id%22%3A5690065567%2C%22_auth_user_backend%22%3A%22accounts.backends.CaseInsensitiveModelBackend%22%2C%22_auth_user_hash%22%3A%22%22%2C%22_token_ver%22%3A2%2C%22_token%22%3A%225690065567%3ASoNBfmOnivtqgH2ER5qBsmEIaPm6gJA8%3A7eca80b7a7592293f0f5896523f35dd3981429b133871bb69e20bef2a4a96309%22%2C%22_platform%22%3A4%2C%22last_refreshed%22%3A1499501508.9412579536%2C%22asns%22%3A%7B%22time%22%3A1499501507%2C%2247.74.12.143%22%3A45102%7D%7D; ig_vw=688; ig_pr=1; csrftoken=wysnnnpLFItwvwi43teN0HYD3bNpLlr0; rur=ASH; ds_user_id=5690065567',
        'referer':'https://www.instagram.com/explore/tags/%E4%BB%A3%E8%B4%AD/',
        'user-agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
        'x-requested-with':'XMLHttpRequest'
    }
    
    z_more = requests.get(url=url_more,params=params,headers=headers_more)
    print z_more.url
    print z_more.status_code
    
    has_next_page = z_more.json()['data']['hashtag']['edge_hashtag_to_media']['page_info']['has_next_page']
    print '是否有下一页'+str(has_next_page)
    
    if has_next_page:
        # 获取新的尾号
        last = '"'+ z_more.json()['data']['hashtag']['edge_hashtag_to_media']['page_info']['end_cursor'] + '"'
        last = last.encode('ascii')
        print '继续加载图片,尾号是'+str(last)
        print '*******************************************************************'
    
    for i in z_more.json()['data']['hashtag']['edge_hashtag_to_media']['edges']:
        print '详情页面的短号是'+str(i['node']['shortcode'])
        shortcode_list.append(i['node']['shortcode'])
        print '------------------------------------------------------------'
        # 通过短号加载详细页面
        # show_text(shortcode = i['node']['shortcode'])
        print "==============================================================="
        count = count + 1
        
    print '已收集%d张图片' %count

https://www.instagram.com/graphql/query/?variables=%7B%22tag_name%22%3A%22%E4%BB%A3%E8%B4%AD%22%2C%22first%22%3A8%2C%22after%22%3A%22J0HWXAaKgAAAF0HWW_E5gAAAFiYA%22%7D&query_id=17875800862117404
200
是否有下一页True
继续加载图片,尾号是"J0HWXAaKgAAAF0HWW-etwAAAFj4A"
*******************************************************************
详情页面的短号是BWvagGJB5sZ
------------------------------------------------------------
详情页面的短号是BWvZgUZA3eu
------------------------------------------------------------
详情页面的短号是BWvSPMBjcTt
------------------------------------------------------------
详情页面的短号是BWvSNgDjD3j
------------------------------------------------------------
详情页面的短号是BWvSMsJD0X5
------------------------------------------------------------
详情页面的短号是BWvSLs6DD00
------------------------------------------------------------
详情页面的短号是BWvSK7GD-ao
------------------------------------------------------------
详情页面的短号是BWvSJFaDNv7
------------------------------------------------------------
详情页面的短号是BWvM4RVgBF6
-------------

https://www.instagram.com/graphql/query/?variables=%7B%22tag_name%22%3A%22%E4%BB%A3%E8%B4%AD%22%2C%22first%22%3A8%2C%22after%22%3A%22J0HWXAaKgAAAF0HWW-HsgAAAFoYBAA%253D%253D%22%7D&query_id=17875800862117404
200
是否有下一页True
继续加载图片,尾号是"J0HWXAaKgAAAF0HWW9puQAAAFp4BAA%3D%3D"
*******************************************************************
详情页面的短号是BWvBG5NAJGb
------------------------------------------------------------
详情页面的短号是BWvA5ChgmeV
------------------------------------------------------------
详情页面的短号是BWu_4gDASDd
------------------------------------------------------------
详情页面的短号是BWu-P_VgQM8
------------------------------------------------------------
详情页面的短号是BWu90W9hT76
------------------------------------------------------------
详情页面的短号是BWu9yiJhtkA
------------------------------------------------------------
详情页面的短号是BWu9xwPBtTZ
------------------------------------------------------------
详情页面的短号是BWu9wTEhplr
------------------------------------------------------------
详情页面的短号是BWu4U

https://www.instagram.com/graphql/query/?variables=%7B%22tag_name%22%3A%22%E4%BB%A3%E8%B4%AD%22%2C%22first%22%3A8%2C%22after%22%3A%22J0HWXAaKgAAAF0HWW8PtQAAAFuYBAA%253D%253D%22%7D&query_id=17875800862117404
200
是否有下一页True
继续加载图片,尾号是"J0HWXAaKgAAAF0HWW7__wAAAFv4BAA%3D%3D"
*******************************************************************
详情页面的短号是BWuG2a7FGJj
------------------------------------------------------------
详情页面的短号是BWuCjCZDBfc
------------------------------------------------------------
详情页面的短号是BWuB9aonQvd
------------------------------------------------------------
详情页面的短号是BWuB75KHHSy
------------------------------------------------------------
详情页面的短号是BWuBLqInPcu
------------------------------------------------------------
详情页面的短号是BWuAtAanlR9
------------------------------------------------------------
详情页面的短号是BWuAXzfnx3L
------------------------------------------------------------
详情页面的短号是BWuAK-LnXZQ
------------------------------------------------------------
详情页面的短号是BWt_3

https://www.instagram.com/graphql/query/?variables=%7B%22tag_name%22%3A%22%E4%BB%A3%E8%B4%AD%22%2C%22first%22%3A8%2C%22after%22%3A%22J0HWXAaKgAAAF0HWW7QagAAAFsYCAA%253D%253D%22%7D&query_id=17875800862117404
200
是否有下一页True
继续加载图片,尾号是"J0HWXAaKgAAAF0HWW7IgwAAAFt4CAA%3D%3D"
*******************************************************************
详情页面的短号是BWtnqLnnltD
------------------------------------------------------------
详情页面的短号是BWtnaJ0nGIT
------------------------------------------------------------
详情页面的短号是BWtmygZHfQk
------------------------------------------------------------
详情页面的短号是BWtmMjIH8UX
------------------------------------------------------------
详情页面的短号是BWtl_5-n9Sn
------------------------------------------------------------
详情页面的短号是BWtluVSHsn8
------------------------------------------------------------
详情页面的短号是BWtljC5H3nH
------------------------------------------------------------
详情页面的短号是BWtlKybHN5w
------------------------------------------------------------
详情页面的短号是BWtkk

SSLError: ("bad handshake: SysCallError(10054, 'WSAECONNRESET')",)

# 四 通过短号获取所要的信息的函数

In [7]:
import js2xml

shortcode = 'BWR-zi9FO9j'

# 获取信息的函数
def show_text(shortcode = shortcode):
    # 通过短号访问图片详细界面
    detailurl = 'https://www.instagram.com/p/'+ shortcode + '/'
    z_detail = requests.get(url = detailurl, headers = headers)
#     print z_detail.url
#     print z_detail.status_code
#     print z_detail.content
    
    # 找出所需的字段信息(如发表图片的用户的用户名,描述图片的文本信息)
    sel_detail = Selector(text = z_detail.text)
    sel_detail.xpath('//script/text()')
#     sel_detail.xpath("//script[contains(., 'window._sharedData")
    
    jscode = sel_detail.xpath("//script[contains(., 'window._sharedData')]/text()").extract_first()
    parsed_js = js2xml.parse(jscode)
#     print js2xml.pretty_print(parsed_js)
    username = parsed_js.xpath('//property[@name="shortcode_media"]/object/property[@name="owner"]/object/property[@name="username"]/string/text()')[0]
    text = parsed_js.xpath('//property[@name="text"]/string/text()')[0]
    
    # 输出用户名及图片文本信息
    print '用户名称为:'+str(username)
    print "    "
    print text
    print '============================================================================================='
    return username,text

# 五 通过短号执行获取信息的函数

In [11]:
count = 0
for i in shortcode_list:
    count = count + 1
    try:
        print '第%d张图片' %count
        print '----------------------------------'
        show_text(shortcode = i.encode('ascii'))
    except BaseException:
        print 'SSLError,无法获取该短号的详情信息'
print "收集完毕,获取了%d张图片的信息" %count

第1张图片
----------------------------------
用户名称为:wonderlanddeluxe
    
#Hermes #愛馬仕 #爱马仕 #愛馬仕包 #爱马仕包 #Hermesbag #handbag #luxury #luxurylife #luxurylifestyle #lifestyle #luxurious #shopaholics #onlineshop #igshop #代購 #代购 #名牌代購 #名牌代购 #歐洲代購 #欧洲代购 #hermes代購 #hermes代购 #愛馬仕代購 #爱马仕代购
如有任何需要歡迎詢問💋
Wechat 📮: wonderlanddeluxe 
What's app 📬: +852 65218289 
Instagram 📷: wonderlanddeluxe
🌏 we do Worldwide Shipping 🌎.
第2张图片
----------------------------------
用户名称为:sharonhong93
    
#woolrich #woolrichparka #woolrich1830 #musthave #coat #multibrand #luxury #instalove #代购
第3张图片
----------------------------------
用户名称为:missbig683
    
#高仿#奢侈品#奢侈品包包#高仿包包#复刻#奢侈品复刻#代购#海关货#原单#A货#名牌#名牌高仿#留学生#正品#一手货源#包包#包
微信：L-aaay
第4张图片
----------------------------------
用户名称为:missbig683
    
#高仿#奢侈品#奢侈品包包#高仿包包#复刻#奢侈品复刻#代购#海关货#原单#A货#名牌#名牌高仿#留学生#正品#一手货源#包包#包
微信：L-aaay
第5张图片
----------------------------------
SSLError,无法获取该短号的详情信息
第6张图片
----------------------------------
用户名称为:missbig683
    
#高仿#奢侈品#奢侈品包包#高仿包包#复刻#奢侈品复刻#代购#海关货#

用户名称为:lijianfeng889
    
#复刻#高仿包包#奢侈品#代购#美女搭配#男士各款百搭款#1比1#礼物#独家实拍#各种新款#原单品质#爱马士#迪奥#GUCCI#香奈儿#普拉达#LV#各种新款奢侈品.专做各种国际奢侈品👜👜，原单品质，🌍全球直邮✈️，诚招全球代理商，💰财富热线💰请加：wechat：1010836568 ❤️Whatsapp：+8617620333773
第31张图片
----------------------------------
用户名称为:lijianfeng889
    
#复刻#高仿包包#奢侈品#代购#美女搭配#男士各款百搭款#1比1#礼物#独家实拍#各种新款#原单品质#爱马士#迪奥#GUCCI#香奈儿#普拉达#LV#各种新款奢侈品.专做各种国际奢侈品👜👜，原单品质，🌍全球直邮✈️，诚招全球代理商，💰财富热线💰请加：wechat：1010836568 ❤️Whatsapp：+8617620333773
第32张图片
----------------------------------
用户名称为:lijianfeng889
    
#复刻#高仿包包#奢侈品#代购#美女搭配#男士各款百搭款#1比1#礼物#独家实拍#各种新款#原单品质#爱马士#迪奥#GUCCI#香奈儿#普拉达#LV#各种新款奢侈品.专做各种国际奢侈品👜👜，原单品质，🌍全球直邮✈️，诚招全球代理商，💰财富热线💰请加：wechat：1010836568 ❤️Whatsapp：+8617620333773
第33张图片
----------------------------------
用户名称为:lijianfeng889
    
#复刻#高仿包包#奢侈品#代购#美女搭配#男士各款百搭款#1比1#礼物#独家实拍#各种新款#原单品质#爱马士#迪奥#GUCCI#香奈儿#普拉达#LV#各种新款奢侈品.专做各种国际奢侈品👜👜，原单品质，🌍全球直邮✈️，诚招全球代理商，💰财富热线💰请加：wechat：1010836568 ❤️Whatsapp：+8617620333773
第34张图片
----------------------------------
用户名称为:lijianfeng889
    
#复刻#高仿包包#奢侈品#代购#美女搭配#男士各款百搭款#1比

用户名称为:insider_outlet
    
💖Celine💖
原價💶1300 EUR
🔔5% off
✔After discount : HKD 11115
香港專門店價格 $14500
👆➡向右划看更多細節
.
下單步驟：
1. 帶圖找Aki
2. 一半訂金找碼/找貨
3a. 有貨：買家付尾數買貨
3b. 無貨：全數退回訂金
4. 貨到香港 順丰到付/面交
.
⚠注意⚠
由於不是人肉帶貨
急單一律不接
到貨時間大概三星期
等不了的請慎重下單
下單後不退不換
鞋子一律無鞋盒
.
#名牌折扣 #名牌 #名牌代購 #名牌代购 #欧洲代购 #巴黎代购 #法国代购 #代购 #法國代購 #歐洲代購 #法國代購 #celine #celinebag #celine代購 #celinecabasphantom
第53张图片
----------------------------------
用户名称为:tina88352
    
很平的宝妈开始大了，还以为是做梦[偷笑]中药调理#减肥#瘦身#丰胸#排毒#祛痘#祛斑#美白#护肤#明星#明星专用#马来西亚#新加坡#台湾#澳洲#英国#美国#代购#全球代购！！了解请加wechat：lili918911
第54张图片
----------------------------------
用户名称为:tina88352
    
很平的宝妈开始大了，还以为是做梦[偷笑]中药调理#减肥#瘦身#丰胸#排毒#祛痘#祛斑#美白#护肤#明星#明星专用#马来西亚#新加坡#台湾#澳洲#英国#美国#代购#全球代购！！了解请加wechat：lili918911
第55张图片
----------------------------------
用户名称为:tina88352
    
很平的宝妈开始大了，还以为是做梦[偷笑]中药调理#减肥#瘦身#丰胸#排毒#祛痘#祛斑#美白#护肤#明星#明星专用#马来西亚#新加坡#台湾#澳洲#英国#美国#代购#全球代购！！了解请加wechat：lili918911
第56张图片
----------------------------------
用户名称为:tina88352
    
很平的宝妈开始大了，还以为是做梦[偷笑]中药调理#减肥#瘦身#丰胸#排毒#祛痘#祛斑#美白#护肤#明星#明星专用#马来西亚#新加

用户名称为:xingmarket
    
.
.
베어리 에코백 베어리 실사에요. 
싱가폴에서 베어리가 나온게 처음이라 물건 빠지는 속도가 엄청 빠르다고 하네요. .
.
구매 하실 분들은 서둘러주시고, 미입금자 분들은 입금전 남은 수량 확인해주세요. .
.
#별마켓#별마트#xingmarket#starbucks#스타벅스구매대행#代购#韩国代购#koreastarbucks #스타벅스#starbuckskeyring #starbuckskeychain #starbucksbear 
#starbuckssingapore#starbucksbearbag#스타벅스베어리#베어리에코백#스타벅스싱가폴#싱가폴스타벅스#싱가폴에코백#스타벅스에코백
第75张图片
----------------------------------
用户名称为:lazy_sis
    
Superme x LV
#代购 #代购正品 
#Supreme
第76张图片
----------------------------------
用户名称为:tina88352
    
同一件衣服穿出不同的效果！！！一盒丸子瘦18斤，衣服能穿上了中药调理#减肥#瘦身#丰胸#排毒#祛痘#祛斑#美白#护肤#明星#明星专用#马来西亚#新加坡#台湾#澳洲#英国#美国#代购#全球代购！！了解请加wechat：lili918911
第77张图片
----------------------------------
用户名称为:tina88352
    
同一件衣服穿出不同的效果！！！一盒丸子瘦18斤，衣服能穿上了中药调理#减肥#瘦身#丰胸#排毒#祛痘#祛斑#美白#护肤#明星#明星专用#马来西亚#新加坡#台湾#澳洲#英国#美国#代购#全球代购！！了解请加wechat：lili918911
第78张图片
----------------------------------
用户名称为:tina88352
    
同一件衣服穿出不同的效果！！！一盒丸子瘦18斤，衣服能穿上了中药调理#减肥#瘦身#丰胸#排毒#祛痘#祛斑#美白#护肤#明星#明星专用#马来西亚#新加坡#台湾#澳洲#英国#美国#代购#全球代购！！了解请加wechat：lili918911
第79张图片
-------

用户名称为:golda_team
    
birkin 25 togo CK10 Craie 奶昔白 金扣 GHW客户订单✈️
#hermes#hermesbirkin#hermesbags
wechat📮:fourbags
instagram📷：golda_team
#代购#代购品质#HK#hongkong#香港🇭🇰#澳门#macao#singapore#新加坡#奢侈品#luxury#luxurylifestyle#bags#
第102张图片
----------------------------------
用户名称为:golda_team
    
birkin 25 togo Q5 Rouge Cossacks 中国红 一直都是非常有人气的颜色 客户订单✈️
#hermes#hermesbirkin#hermesbags
wechat📮:fourbags
instagram📷：golda_team
#代购#代购品质#HK#hongkong#香港🇭🇰#澳门#macao#singapore#新加坡#奢侈品#luxury#luxurylifestyle#bags#
第103张图片
----------------------------------
用户名称为:golda_team
    
birkin 25 togo CK18
Etoupe 大象灰 银扣 低调有品味的颜色 适合任何一个年龄的颜色 客户订单✈️
#hermes#hermesbirkin#hermesbags
wechat📮:fourbags
instagram📷：golda_team
#代购#代购品质#HK#hongkong#香港🇭🇰#澳门#macao#singapore#新加坡#奢侈品#luxury#luxurylifestyle#bags#
第104张图片
----------------------------------
用户名称为:golda_team
    
birkin 25 togo CK37 Gold 金棕色 金扣 GHW超级好看百搭色 夏天小包超级受欢迎 
#hermes#hermesbirkin#hermesbags
wechat📮:fourbags
instagram📷：golda_team
#代购#代购品质#HK#hongkong#香港🇭🇰#澳门#macao#si

用户名称为:ouzhouzhan1781
    
#代购#香港#澳门#美国#马来西亚#台湾#英国#澳洲#加拿大#新加坡#全球代购#韩国代购#泰国代购#女朋友#包包#名牌#英国代购#美国代购#德国#迪拜#法国#上海#北京#广州#深圳
微信：OZZ-1989
第127张图片
----------------------------------
用户名称为:ouzhouzhan1781
    
#代购#香港#澳门#美国#马来西亚#台湾#英国#澳洲#加拿大#新加坡#全球代购#韩国代购#泰国代购#女朋友#包包#名牌#英国代购#美国代购#德国#迪拜#法国#上海#北京#广州#深圳
微信：OZZ-1989
第128张图片
----------------------------------
用户名称为:golda_team
    
H Mini kelly  鸵鸟皮 糖果粉色 银扣
#hermes#hermeskelly#hermesbags
wechat📮:fourbags
instagram📷：golda_team
#代购#代购品质#HK#hongkong#香港🇭🇰#澳门#macao#singapore#新加坡#奢侈品#luxury#luxurylifestyle#bags#
第129张图片
----------------------------------
用户名称为:golda_team
    
H Mini kelly  鸵鸟皮 黑色 银扣
#hermes#hermeskelly#hermesbags
wechat📮:fourbags
instagram📷：golda_team
#代购#代购品质#HK#hongkong#香港🇭🇰#澳门#macao#singapore#新加坡#奢侈品#luxury#luxurylifestyle#bags#
第130张图片
----------------------------------
用户名称为:golda_team
    
H Mini kelly  鸵鸟皮 黑色 银扣 
#hermes#hermeskelly#hermesbags
wechat📮:fourbags
instagram📷：golda_team
#代购#代购品质#HK#hongkong#香港🇭🇰#澳门#macao#singapore#新加坡#奢侈品#lu

用户名称为:golda_team
    
Bearn 两折钱包Epsom CK 18 Etoupe大象灰 金扣 GHW 经典颜色
#hermes#hermes钱包#hermesbags
wechat📮:fourbags
instagram📷：golda_team
#代购#代购品质#HK#hongkong#香港🇭🇰#澳门#macao#singapore#新加坡#奢侈品#luxury#luxurylifestyle#bags#
第151张图片
----------------------------------
用户名称为:golda_team
    
Bearn 两折钱包Epsom CK 18 Etoupe大象灰 银扣 GHW 经典颜色 
#hermes#hermes钱包#hermesbags
wechat📮:fourbags
instagram📷：golda_team
#代购#代购品质#HK#hongkong#香港🇭🇰#澳门#macao#singapore#新加坡#奢侈品#luxury#luxurylifestyle#bags#
第152张图片
----------------------------------
用户名称为:golda_team
    
Bearn 黑金两折钱包 CK89 Nior 刚出货10个 
#hermes#hermes钱包#hermesbags
wechat📮:fourbags
instagram📷：golda_team
#代购#代购品质#HK#hongkong#香港🇭🇰#澳门#macao#singapore#新加坡#奢侈品#luxury#luxurylifestyle#bags#
第153张图片
----------------------------------
用户名称为:xiaolaohu3333_
    
#减肥#最爱#减肥药#留学生#美国#微整形#我的前半生#微信#土豪#网红#香港#paypal#瘦#健身#dior#chanel#么么哒#美白#代购#海外#台湾#英国#韩国#CL#

微信：xiaolaohu3333✌️
第154张图片
----------------------------------
用户名称为:xiaolaohu3333_
    
#减肥#最爱#减肥药#留学生#美国#微整形#我的前半生#微信#土豪

用户名称为:aimeilizhan
    
#正品代购#原单品质#高仿包包#街景#顶级品质#礼物#美女搭配#奢侈品#代购#品牌背包#炫耀#A货#贝壳背包#海关货#原单货#货源背包#品牌皮具#奢侈酒吧#一比一#高仿鞋#独家实物拍摄#  只做市场最好版本！独家实物拍摄，全球直邮。诚招代理！ ❤️请加微信wechat：👠👉aimeilizhan 👜👉chao784254262 ❤️Whatsapp：+8615218851609
第178张图片
----------------------------------
用户名称为:aimeilizhan
    
#正品代购#原单品质#高仿包包#街景#顶级品质#礼物#美女搭配#奢侈品#代购#品牌背包#炫耀#A货#贝壳背包#海关货#原单货#货源背包#品牌皮具#奢侈酒吧#一比一#高仿鞋#独家实物拍摄#  只做市场最好版本！独家实物拍摄，全球直邮。诚招代理！ ❤️请加微信wechat：👠👉aimeilizhan 👜👉chao784254262 ❤️Whatsapp：+8615218851609
第179张图片
----------------------------------
用户名称为:aimeilizhan
    
#正品代购#原单品质#高仿包包#街景#顶级品质#礼物#美女搭配#奢侈品#代购#品牌背包#炫耀#A货#贝壳背包#海关货#原单货#货源背包#品牌皮具#奢侈酒吧#一比一#高仿鞋#独家实物拍摄#  只做市场最好版本！独家实物拍摄，全球直邮。诚招代理！ ❤️请加微信wechat：👠👉aimeilizhan 👜👉chao784254262 ❤️Whatsapp：+8615218851609
第180张图片
----------------------------------
用户名称为:aimeilizhan
    
#正品代购#原单品质#高仿包包#街景#顶级品质#礼物#美女搭配#奢侈品#代购#品牌背包#炫耀#A货#贝壳背包#海关货#原单货#货源背包#品牌皮具#奢侈酒吧#一比一#高仿鞋#独家实物拍摄#  只做市场最好版本！独家实物拍摄，全球直邮。诚招代理！ ❤️请加微信wechat：👠👉aimeilizhan 👜👉chao784254262 ❤️Whatsapp：+8615218851609
第181张图片
-------

In [12]:
show_text(shortcode = 'BWt_PKwH-Hh')

用户名称为:golda_team
    
birkin 30 epsom ck37 blue saphir 宝石蓝 金扣 客户订单✈️
#hermes#hermesbirkin#hermesbags
wechat📮:fourbags
instagram📷：golda_team
#代购#代购品质#HK#hongkong#香港🇭🇰#澳门#macao#singapore#新加坡#奢侈品#luxury#luxurylifestyle#bags#


('golda_team',
 u'birkin 30 epsom ck37 blue saphir \u5b9d\u77f3\u84dd \u91d1\u6263 \u5ba2\u6237\u8ba2\u5355\u2708\ufe0f\n#hermes#hermesbirkin#hermesbags\nwechat\U0001f4ee:fourbags\ninstagram\U0001f4f7\uff1agolda_team\n#\u4ee3\u8d2d#\u4ee3\u8d2d\u54c1\u8d28#HK#hongkong#\u9999\u6e2f\U0001f1ed\U0001f1f0#\u6fb3\u95e8#macao#singapore#\u65b0\u52a0\u5761#\u5962\u4f88\u54c1#luxury#luxurylifestyle#bags#')

# 正则表达式

In [13]:
import re

In [14]:
text = show_text(shortcode = 'BWt_PKwH-Hh')[1]

用户名称为:golda_team
    
birkin 30 epsom ck37 blue saphir 宝石蓝 金扣 客户订单✈️
#hermes#hermesbirkin#hermesbags
wechat📮:fourbags
instagram📷：golda_team
#代购#代购品质#HK#hongkong#香港🇭🇰#澳门#macao#singapore#新加坡#奢侈品#luxury#luxurylifestyle#bags#


In [15]:
print text

birkin 30 epsom ck37 blue saphir 宝石蓝 金扣 客户订单✈️
#hermes#hermesbirkin#hermesbags
wechat📮:fourbags
instagram📷：golda_team
#代购#代购品质#HK#hongkong#香港🇭🇰#澳门#macao#singapore#新加坡#奢侈品#luxury#luxurylifestyle#bags#


In [18]:
pattern = re.compile(r'wechat')
for i in text.split():
    print i
    match = pattern.match(i)
    if match:
        print '找到wechat信息'
        print match.group()

birkin
30
epsom
ck37
blue
saphir
宝石蓝
金扣
客户订单✈️
#hermes#hermesbirkin#hermesbags
wechat📮:fourbags
找到wechat信息
wechat
instagram📷：golda_team
#代购#代购品质#HK#hongkong#香港🇭🇰#澳门#macao#singapore#新加坡#奢侈品#luxury#luxurylifestyle#bags#


In [33]:
for i in text.split():
#     print i
    find_wechat(i)
    find_whatsapp(i)

找到wechat信息
wechat📮:fourbags


### 查找指定信息

In [30]:
#short_text text片段
def find_wechat(short_text):
    pattern_wechat = re.compile(r'wechat')
    pattern_wechat_ch = re.compile(r'微信')
    match = pattern_wechat.match(short_text)
    if match:
        print '找到wechat信息'
        print i
    match = pattern_wechat_ch.match(short_text)
    if match:
        print '找到wechat信息'
        print i

In [32]:
#short_text text片段
def find_whatsapp(short_text):
    count = 0
    pattern_whatsapp = re.compile(r'whatsapp')
    pattern_wts = re.compile(r'wts')
    match = pattern_whatsapp.match(short_text)
    if match:
        print '找到whatsapp信息,keyword is whatsapp'
        print i
        count = count + 1
    match = pattern_wts.match(short_text)
    if match:
        print '找到wechat信息, keyword is wts'
        print i
        count = count + 1
#     if count == 0:
#         print "找不到whatsapp信息"

In [53]:
def find_whatsapp_number(short_text):
    pattern_whatsapp_number = re.compile(r'<[^\D]>\d+<[^\D]>')
    pattern_whatsapp_number = re.compile(r'([0-9]*)')
    
    p = re.compile(r'\d{3}-\d{6}')
    
    match = pattern_whatsapp_number.match(short_text)
    if match:
        print '找到whatsapp号码信息,keyword is numbers'
        print match.group()

In [54]:
text = "adfadfa12312334adfadfafdasfdasdfs"
print text
find_whatsapp_number(text)

adfadfa12312334adfadfafdasfdasdfs
adfadfa12312334adfadfafdasfdasdfs
找到whatsapp号码信息,keyword is numbers

